In [49]:
# What things can you do on your turn in Hanabi?
# 1. Play a card from your hand. -- Choose to play, choose a card.
# 2. Discard a card from your hand. -- Choose to discard, choose a card.
# 3. Give a hint to the other player about cards in their hand. -- Choose to give hint, choose a hint.

# What information do you have at any point in the Hanabi Game?
# 1. Hints given for each of your cards.
# 2. Number of remaining hints.
# 3. Number of cards left in the deck.
# 4. Number of lives left.
# 5. Hints given to other players already.
# 6. Cards in play.
# 7. Score.

# Maybe we can describe the state by [[score], [lives remaining], [hints available], [# cards in deck],
#                                     [Cards in play], [Hints in Hand], [Other People's Hands Hints]]

In [78]:
# We will have a game manager script that takes in a list of players and sends them the game state when it is their turn.
# We then instantiate a couple of players and have them play games trying to achieve the highest score.

In [128]:
import itertools
import random

class HanabiGame:
    
    def __init__(self, hand_size = 5, hints = 8, lives = 3, num_players = 2, num_suits = 5, num_values = 5):
                    
        # Hand Size
        if hand_size < 1:
            print("How are you going to play without any cards in your hand, silly?")
        self.hand_size = hand_size
        
        # Hints remaining
        self.hints = hints
        self.max_hints = self.hints
        
        # Lives remaining
        self.lives = lives

        # Number of players
        if num_players < 2:
            print("Hanabi must be played with at least two players.")
            
        self.num_players = num_players
        
        # Suits of Cards
        self.num_suits = num_suits
        self.suits = list(range(self.num_suits))

        # Values of Cards
        # There are 3 zeros of each suit, one of the maximal value, and two of all others.
        if num_values < 2:
            print("Please allow for at least two values.")
        
        self.num_values = num_values
        self.values = [0, 0, 0] + list(range(1, self.num_values-1))*2 + [self.num_values-1]

        # Deck
        # A card in the deck's ten's place is the suit of the card and 
        # the one's place is the value
        self.deck = [(suit,value) for suit, value in itertools.product(self.suits,self.values)]

        # Cards in the Hands of each Player
        self.player_hands = [[] for i in range(num_players)]
        self.initialDeal()

        # Hint information available to each player
        # self.player_hints[0] = [Hints received for suits, Hints received for values, turns_in_hand]
        # Hints received are either 0 (no info), 1 (positive info), or -1 (negative info)
        # So, [[-1, 0, 0, 0, 0], [-1, 1, -1, -1, -1], 3] would correspond to knowing a card is
        # NOT of suit 0 and IS of value 1 and has been in your hand for 3 turns.
        self.player_hints = [[[[0]*self.num_suits,[0]*self.num_values, 0] for j in range(self.hand_size)]
                             for i in range(self.num_players)]
        
        # Keep track of whose turn it is
        self.current_player_turn = 0
        
        # Keep track of the score.
        # Note*: Normally in Hanabi your score is zero until the last turn
        # is played, but to help the AI, the current score will only become
        # zero when all lives are lost.
        self.current_score = 0
        
        # Cards that have been played onto the board
        # -1 corresponds to no card of a particular suit having been played.
        # The current value of self.board[i] = Highest value of card played
        # of that suit.
        self.board = [-1]*self.num_suits
        
        # Discard Pile
        # self.discards[i][j] = The number of suit i value j cards that have been discarded so far.
        self.discards = [[0]*num_values for i in range(self.num_suits)]
        
        # Keep a list of possible next moves
        self.legal_move_list = []
        # Initialize legal_move_list
        self.legalMoves()
        
        # Keep track of the number of moves left when the final round starts
        self.final_round_moves = self.num_players
        self.game_is_ending = False
        
        # When the game ends, the game manager should deal with cleaning everything up.
        self.game_end = False


    def initialDeal(self):
        random.shuffle(self.deck)
        for i in range(self.hand_size):
            for hand in range(self.num_players):
                self.player_hands[hand].append(self.deck.pop())
    
    # Returns the board state from a given player's perspective.
    # In particular, it returns all public information and the cards in the opponents hands.
    # [[score]: 1, [lives remaining]: 1, [hints available]: 1, [# cards in deck]: 1,
    # [Cards in play]: num_suits, [Discards]: num_suits*num_values, [Hints in Hand], [Other People's Hands Hints]]
    def boardState(self, player_number):
        return [self.current_score, self.lives, self.hints, len(self.deck), self.board, self.discards, 
                self.player_hints[player_number]] + [hint for i,hint in enumerate(self.player_hints) if i!= player_number]
    
    def printBoardState(self, player_number=-1):
        
        # If no player is specified, return the view from the current player
        if player_number == -1:
            player_number = self.current_player_turn
#         cur_board = boardState(player_number)
        print(f"From the perspective of player {player_number}")
        print(f"The current score is {self.current_score}.")
        print(f"You have {self.lives} lives and {self.hints} hints left.")
        print(f"There are {len(self.deck)} cards left in the deck.")
        for i, max_card in enumerate(self.board):
            if max_card == -1:
                print(f"You have not played any cards of suit {i}")
            elif max_card == 0:
                print(f"You have played the 0 of suit {i}.")
            else:
                print(f"You have played {max_card} cards of suit {i}.")
        
        for card_index, hint in enumerate(self.player_hints[player_number]):
            if hint == [[0]*self.num_suits, [0]*self.num_values, 0]:
                print(f"You don't know anything about card {card_index}.")
            else:
                if hint[0] == [0]*self.num_suits:
                    print(f"You don't know anything about the suit of card {card_index}.")
                else:
                    if 1 in hint[0]:
                        print(f"You know that {card_index} is of suit {(hint[0]).index(1)}")
                    else:
                        for suit_index, existence in enumerate(hint[0]):
                            if existence == -1:
                                print(f"You know card {card_index} is not of suit {suit_index}.")
                
                if hint[1] == [0]*self.num_values:
                    print(f"You don't know anything about the value of card {card_index}.")
                else:
                    if 1 in hint[1]:
                        print(f"You know that {card_index} is of suit {(hint[0]).index(1)}")
                    else:
                        for value_index, existence in enumerate(hint[1]):
                            if existence == -1:
                                print(f"You know card {card_index} is not of value {value_index}.")

                            
    # Returns a tuple (The index of the current player's turn, [List of allowable moves])    
    def legalMoves(self):
        
        # Moves will be described by a tuple 
        # (Choose to Play/Discard/Hint, Index of Card to Discard/Play [-1 if giving hint instead], 
        #                              Hint Type[(player, suit, value)])
        # Hint Type is (-1,-1,-1) if not giving a hint.
        allowed_moves = []
        
        # Play moves (0,X,-1,-1)
        for i in range(len(self.player_hands[self.current_player_turn])):
            allowed_moves.append((0,i,-1,-1,-1))
        
        # Discard moves (1,X,-1,-1)
        for i in range(len(self.player_hands[self.current_player_turn])):
            allowed_moves.append((1,i,-1,-1,-1))
        
        
        # If hint available, hint moves (2, -1, Player, Suit, Value)
        if self.hints > 0:
            for player_index in range(self.num_players):
                if player_index != self.current_player_turn:
                    for suit_index in range(self.num_suits):
                        allowed_moves.append((2,-1,player_index,suit_index,-1))
                    for value_index in range(self.num_values):
                        allowed_moves.append((2,-1,player_index,-1,value_index))
        
        self.legal_move_list = allowed_moves
        return (self.current_player_turn, allowed_moves)
    
    def printLegalMoves(self):
        self.legalMoves()
        print(f"It is Player {self.current_player_turn}'s turn.")
        if len(self.legal_move_list) == 0:
            print("You done goofed. No legal moves.")
            return
        for index, move in enumerate(self.legal_move_list):
            if move[0] == 0:
                print(f"{index}: You can play your {move[1]} card.")
            elif move[0] == 1:
                print(f"{index}: You can discard your {move[1]} card.")
            else:
                if move[3] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of suit {move[3]}.")
                elif move[4] != -1:
                    print(f"{index}: You can tell player {move[2]} about their cards of value {move[4]}.")
                else:
                    print('We definitely should not have gotten here.')
    
    # Current Player Draws a card
    def drawCard(self):
        if len(self.deck) == 0:
            self.game_is_ending = True
            self.final_round_moves -= 1
            if self.final_round_moves == 0:
                self.endGame()

        if len(self.player_hands[self.current_player_turn]) != 4:
            print(f"A player is trying to draw with {len(self.player_hands[self.current_player_turn])} cards in their hand.")
            return
        else:
            self.player_hands[self.current_player_turn].append(self.deck.pop())
            self.player_hints[self.current_player_turn].append([[0]*self.num_suits,[0]*self.num_values, 0])
        
        self.legalMoves()
    
    # Implement this
    def endGame(self):
        self.game_end = True
        return
    
    def playCard(self, index):
        if index > len(self.player_hands[self.current_player_turn]):
            print("That card is unplayable. Uh oh.")
            return
        
        card = self.player_hands[self.current_player_turn][index]
        suit = card[0]
        value = card[1]
        
        
        # If the card is playable, play it, increment score, and draw a card.
        # Otherwise, we'll lose a life, check game end, discard it, and draw a card.
        if self.board[suit] + 1 == value:
            self.board[suit] = value
            self.current_score += 1
        else:
            self.lives -= 1
            if self.lives <= 0:
                endGame()
                return
        self.discardCard(index, gain_hint = False)
        self.drawCard()

    # Only draw a card if gain_hint = True
    # Update discard pile
    def discardCard(self, index, gain_hint = True):
        self.discards[self.player_hands[self.current_player_turn][index][0]][
                            self.player_hands[self.current_player_turn][index][1]]+= 1
        del self.player_hands[self.current_player_turn][index]
        del self.player_hints[self.current_player_turn][index]
        if gain_hint == True:
            self.drawCard()
            if self.hints < self.max_hints:
                self.hints += 1
        return

    # If a hint is given whilst game is ending, remember to decrement final_round_moves
    # self.player_hints = [[[[0]*self.num_suits,[0]*self.num_values] for j in range(self.hand_size)]
    #                        for i in range(self.num_players)]
    def giveHint(self, hint):
        player, suit_hint, value_hint = hint[0], hint[1], hint[2]
        
        if self.hints <= 0:
            print("You cannot give a hint, as you have no hint tokens left!")
        
        # card_hints is of the form [[suit hints], [value hints]]
        for card_index, card_hints in enumerate(self.player_hints[player]):
            card_suit = self.player_hands[player][card_index][0]
            card_value = self.player_hands[player][card_index][1]
            if (suit_hint != -1):
                if suit_hint == card_suit:
                    card_hints[0] = [1 if x == card_suit else -1 for x in range(self.num_suits) ]
                else:
                    card_hints[0][suit_hint] = -1
            elif (value_hint != 1):
                if value_hint == card_value:
                    card_hints[1] = [1 if x == card_value else -1 for x in range(self.num_values) ]
                else:
                    card_hints[1][suit_hint] = -1
            else:
                print("We were given neither a suit nor a value.")
                return
        self.hints -= 1
        return

    
    def performMove(self, index = -1, move = None):
        if index == -1 and move == None:
            print("What move would you like to do?")
            return
        if index > len(self.legal_move_list):
            print("Choose a valid move. [Index out of range.]")
            return
        if move != None and move not in legal_move_list:
            print("Choose a valid move. [Move not legal.]")
            return

        if index != -1:
            move = self.legal_move_list[index]
#         else:
#             index = legal_move_list.index(move)
        
        # Reminder: move = (play/discard/hint, card to play/discard index, player to give hint, suit hint, value hint)
        # Reminder: move[0]: 0 - Play, 1 - Discard, 2 - Hint
        if move[0] == 0:
            self.playCard(move[1])
        elif move[0] == 1:
            self.discardCard(move[1])
        elif move[0] == 2:
            self.giveHint((move[2], move[3], move[4]))
        
        # Next player's turn
        self.legalMoves()
        for card in self.player_hints[self.current_player_turn]:
            # Increment the time the card has been in the player's hand
            card[2] += 1
        self.current_player_turn = (self.current_player_turn + 1) % self.num_players
        

In [69]:
test_game = HanabiGame()

In [70]:
test_game.printBoardState()

From the perspective of player 0
The current score is 0.
You have 3 lives and 8 hints left.
There are 40 cards left in the deck.
You have not played any cards of suit 0
You have not played any cards of suit 1
You have not played any cards of suit 2
You have not played any cards of suit 3
You have not played any cards of suit 4
You don't know anything about card 0.
You don't know anything about card 1.
You don't know anything about card 2.
You don't know anything about card 3.
You don't know anything about card 4.


In [71]:
test_game.printLegalMoves()

It is Player 0's turn.
0: You can play your 0 card.
1: You can play your 1 card.
2: You can play your 2 card.
3: You can play your 3 card.
4: You can play your 4 card.
5: You can discard your 0 card.
6: You can discard your 1 card.
7: You can discard your 2 card.
8: You can discard your 3 card.
9: You can discard your 4 card.
10: You can tell player 1 about their cards of suit 0.
11: You can tell player 1 about their cards of suit 1.
12: You can tell player 1 about their cards of suit 2.
13: You can tell player 1 about their cards of suit 3.
14: You can tell player 1 about their cards of suit 4.
15: You can tell player 1 about their cards of value 0.
16: You can tell player 1 about their cards of value 1.
17: You can tell player 1 about their cards of value 2.
18: You can tell player 1 about their cards of value 3.
19: You can tell player 1 about their cards of value 4.


In [72]:
test_game.performMove(index = 10)

In [73]:
test_game.printLegalMoves()

It is Player 1's turn.
0: You can play your 0 card.
1: You can play your 1 card.
2: You can play your 2 card.
3: You can play your 3 card.
4: You can play your 4 card.
5: You can discard your 0 card.
6: You can discard your 1 card.
7: You can discard your 2 card.
8: You can discard your 3 card.
9: You can discard your 4 card.
10: You can tell player 0 about their cards of suit 0.
11: You can tell player 0 about their cards of suit 1.
12: You can tell player 0 about their cards of suit 2.
13: You can tell player 0 about their cards of suit 3.
14: You can tell player 0 about their cards of suit 4.
15: You can tell player 0 about their cards of value 0.
16: You can tell player 0 about their cards of value 1.
17: You can tell player 0 about their cards of value 2.
18: You can tell player 0 about their cards of value 3.
19: You can tell player 0 about their cards of value 4.


In [74]:
test_game.printBoardState()

From the perspective of player 1
The current score is 0.
You have 3 lives and 7 hints left.
There are 40 cards left in the deck.
You have not played any cards of suit 0
You have not played any cards of suit 1
You have not played any cards of suit 2
You have not played any cards of suit 3
You have not played any cards of suit 4
You know card 0 is not of suit 0.
You don't know anything about the value of card 0.
You know card 1 is not of suit 0.
You don't know anything about the value of card 1.
You know that 2 is of suit 0
You don't know anything about the value of card 2.
You know that 3 is of suit 0
You don't know anything about the value of card 3.
You know card 4 is not of suit 0.
You don't know anything about the value of card 4.


In [75]:
test_game.discards

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [ ]:
# Implementation of Game Manager that will run games for the player AI and provide training

In [145]:
def flatten(list_to_flatten):
        found_something_to_flatten = True
        while(found_something_to_flatten == True):
            found_something_to_flatten = False
            temp_list = []
            for i, item in enumerate(list_to_flatten):
                if isinstance(item, list):
                    found_something_to_flatten = True
                    temp_list += list_to_flatten[i]
                else:
                    temp_list.append(list_to_flatten[i])
            list_to_flatten = temp_list
        return list_to_flatten
    
test_game = HanabiGame()

flat_state = flatten(test_game.boardState(0))
input_size = len(flat_state) # This equals 144 in 2 player normal game
print(f"The input length is {input_size}.")

# Genetic Algorithm to find a good HanabiAI
# Initialize with 2*N players (Initial Population)

population = 2*1
# The weights in this current implementation are 10,880
num_weights = 10880
current_generation_players = []

for i in range(population):
    # [Player, score_achieved]
    current_generation_weights.append([HanabiPlayer(game_state_size=input_size,weights=np.random.rand(10880)), 0])


    
# Evalute the fitness of each player (which we'll consider to be the maximum score obtained before game ends.)

# I am going to implement it now where pairs of players play different games, but maybe it'd be better 
# to have all pairs of players play through the same game for fairness. If they happen to play an easy game
# They might get a bias.

#How many times will we train the group?
num_gen_to_train = 2

for i in range(num_gen_to_train):
    # Sort randomly and take pairs in order as game partners
    random.shuffle(current_generation_players)
    for j in range(int(population/2)):
        cur_game = HanabiGame()
        while cur_game.game_end == False:
            print(i,j)
            break

# Generate a new generation by randomly choosing two* parents biased upon fitness
# Create a child with new weights based upon the cross-over of parents
# Mutate with some small chance
# Repeat

# Save weights after training? Or maybe save N of them?






The input length is 144.
0 0
1 0


In [139]:
A =[1,2,3]
random.shuffle(A);A

[1, 3, 2]

In [ ]:
# Implementation of Player AI

In [129]:
import numpy as np
import random
import statistics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

class HanabiPlayer:
    
    # Game state will be ordered as
    # [[score]: 1, [lives remaining]: 1, [hints available]: 1, [# cards in deck]: 1,
    # [Cards in play]: num_suits, [Discards]: num_suits*num_values, [Hints in Hand], [Other People's Hands Hints]]
    def __init__(self, game_state_size, weights, num_players = 2):
        
        self.game_state_size = game_state_size
        self.num_players = 2
        
        # These are random. Just a guess!
        second_layer_size = 64
        third_layer_size = 16
        
        # Allows for modular layer sizes
        # The final layer is the number of possible moves to output.
        # There are always hand_size*2 (plays or discards) + num_opponents * (num_values + num_suits) (hints)
        # possible moves. FOR NOW -- We're gonna assume normal hand size with 5 suits and 5 values for convenience.
        # Hence, 5*2 + (num_players - 1)*10
        self.layer_sizes = [second_layer_size, third_layer_size, 10+10*(self.num_players-1)]
        
        self.model = Sequential([
            # Input will be a vector of game_state_size length
            Dense(units=self.layer_sizes[0], input_shape=(game_state_size,), use_bias=False, activation='relu'),
            Dense(units=self.layer_sizes[1], use_bias=False, activation='relu'),
            Dense(units=self.layer_sizes[2], use_bias=False, activation='sigmoid')
        ])
        
#         if any(weights == None):
#             self.weights = np.array([0]*((game_state_size)*64+64*16+16*20))
#         else:
        
        self.set_weights(weights)
    
        # Since we will be updating the weights ourselves, this isn't necessary...
        self.model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        
        
    # Set the given weights for the model
    def set_weights(self, np_flat_weights):
        if len(np_flat_weights) != (self.game_state_size*self.layer_sizes[0]+
                                    self.layer_sizes[0]*self.layer_sizes[1]+
                                    self.layer_sizes[1]*self.layer_sizes[2]):
            weights0 = np_flat_weights[:self.game_state_size*self.layer_sizes[0]].copy()
            weights1 = np_flat_weights[self.game_state_size*self.layer_sizes[0]:
                                      self.layer_sizes[0]*self.layer_sizes[1]].copy()
            weights2 = np_flat_weights[self.layer_sizes[0]*self.layer_sizes[1]:
                                      self.layer_sizes[1]*self.layer_sizes[2]].copy()
            
            weights0.resize(self.game_state_size,self.layer_sizes[0])
            weights1.resize(self.layer_sizes[0],self.layer_sizes[1])
            weights2.resize(self.layer_sizes[1],self.layer_sizes[2])

            self.model.layers[0].set_weights([weights0])
            self.model.layers[1].set_weights([weights1])
            self.model.layers[2].set_weights([weights2])
            
            return
        
    
    def choose_best_move(self, move_list, game_state):
        
        neural_input = np.asarray([])
        return
    
    def update_game(self, game_state):
        self.game_state = game_state
        return
    
    # Genetic Algorithm Stuff
    def dna_crossover(self, partners):
        partners.append(self.weights)
        average_weights = []
        for i in range(len(self.weights)):
            average_weights.append(statistics.mean([player_weight[i] for player_weight in partners]))
        return self.mutate(average_weights)
    
    def mutate(self, dna_weights):
        mutation_rate = 0.01
        new_weights = []
        
        for weight in dna_weights:
            if random.random() < mutation_rate:
                new_weights.append(random.random())
            else:
                new_weights.append(weight)
        
        return new_weights

# test_game = HanabiGame()
# input_size = len(flatten(test_game.boardState(0)))+len(test_game.legal_move_list[0])
# print(input_size)
# test_player = HanabiPlayer(game_state_size = input_size)
# # len(board_state) = 4 + num_suits + num_suits*num_values + hand_size*(num_suits+num_values+1)*num_players + len(move)
# # In 2 player, 5 cards, 5 suits, 5 values, you should get 144
# weights = test_player.model.get_weights()
# test_player.model.summary()

In [71]:
np.array([0,0])

array([0, 0])

In [54]:
test_player.model.layers[0].get_weights()[0].shape

(149, 64)

In [56]:
test_player.model.layers[0].get_weights()[0]

array([[ 0.08083725, -0.16372919,  0.13646606, ...,  0.10873467,
        -0.07031066,  0.14976054],
       [-0.0698402 ,  0.07548264,  0.02912329, ...,  0.0288291 ,
        -0.15149724,  0.06777997],
       [-0.15602344, -0.06794512, -0.02990806, ..., -0.04343148,
         0.02203298, -0.07477309],
       ...,
       [-0.16625635, -0.09392204,  0.00844222, ..., -0.08807269,
         0.04357317,  0.15217835],
       [-0.16565296, -0.10612939,  0.1478456 , ...,  0.13770753,
         0.09658301,  0.15465763],
       [ 0.09059453,  0.0597778 ,  0.12578863, ...,  0.06466271,
         0.07777975,  0.1496864 ]], dtype=float32)

In [55]:
new_weights = np.random.rand(1,149)
new_bias = np.random.rand(149)
print(new_weights, new_bias)

[[0.42339469 0.59072736 0.04346396 0.36921308 0.57324186 0.47863045
  0.45791628 0.47804995 0.16343983 0.30044142 0.57172537 0.00194719
  0.84115561 0.81339363 0.19973684 0.4461811  0.32571168 0.44035575
  0.83788181 0.16891712 0.0667543  0.31636646 0.26248435 0.98899494
  0.21391741 0.39581786 0.10046035 0.39308135 0.02301789 0.79999569
  0.71346973 0.00156998 0.56385111 0.61771368 0.01113974 0.39016874
  0.76070422 0.96651385 0.37124463 0.85165317 0.55980967 0.58968647
  0.62261158 0.0851347  0.660229   0.88334776 0.11124298 0.68901369
  0.8705029  0.50224778 0.29646481 0.28232894 0.18858136 0.98203882
  0.83226869 0.6535438  0.16350148 0.86605551 0.8586029  0.61741073
  0.67374615 0.40780479 0.49670143 0.40764699 0.01630312 0.42643567
  0.85709073 0.61993484 0.92824184 0.18155639 0.20618189 0.89820398
  0.99704972 0.63952065 0.88343214 0.27642614 0.32408112 0.79657105
  0.7250619  0.64232649 0.93042271 0.32974205 0.17189262 0.35596026
  0.55618476 0.02335208 0.05308243 0.359221   0.

In [35]:
test_player.model.layers[0].set_weights([new_weights,new_bias])

In [36]:
test_player.model.layers[0].get_weights()

[array([[0.74102145, 0.849025  , 0.04187267, 0.7883437 , 0.01192012,
         0.5243464 , 0.06813269, 0.96124667, 0.33558008, 0.93164283,
         0.588623  , 0.67266136, 0.45087466, 0.51839036, 0.66178864,
         0.09762914, 0.14006124, 0.94946045, 0.94838935, 0.6218719 ,
         0.9905013 , 0.25324613, 0.01156903, 0.52270705, 0.08498961,
         0.8288506 , 0.8663491 , 0.4742032 , 0.16545579, 0.288636  ,
         0.38427716, 0.41033298, 0.5384884 , 0.14657351, 0.64696455,
         0.04869553, 0.40379876, 0.04056195, 0.67256176, 0.66531813,
         0.60797656, 0.00682958, 0.44513407, 0.04815576, 0.42611706,
         0.6989127 , 0.5338269 , 0.19528802, 0.39488664, 0.80430424,
         0.7494596 , 0.68400526, 0.8781394 , 0.71430045, 0.33782193,
         0.9610376 , 0.90094477, 0.978812  , 0.37901828, 0.26521376,
         0.40755296, 0.43177605, 0.61608744, 0.9786455 , 0.9256213 ,
         0.7425722 , 0.9052971 , 0.15342563, 0.6995551 , 0.13974942,
         0.29918578, 0.01148409, 0

In [45]:
for i in range(len(test_player.model.layers)):
    print(i, test_player.model.layers[i].get_weights()[0].shape)

0 (1, 149)
1 (149, 64)
2 (64, 16)
3 (16, 1)


In [ ]:
[i for i in range(10) if i!=2]

In [ ]:
print(f"Is{' not' if True else ''}!")

In [ ]:
A = [[1,2],[3,4]]
for a in A:
    a[0] = 0
A

In [22]:
A = [0,0,1]
B = ['a' if x == 0 else 'b' for x in A ]

In [23]:
B

['a', 'a', 'b']

In [158]:
random.random()

0.42255412843089846

In [82]:
flatten([[[1,2],[3,4]],[[5,6],[7,8]]])

[1, 2, 3, 4, 5, 6, 7, 8]

In [80]:
[1,2] + [1,2,3]

[1, 2, 1, 2, 3]

In [157]:
A = [1]
A.append([[1,2],[3,4],[5,6]])
print(A)

[1, [[1, 2], [3, 4], [5, 6]]]


In [46]:
input_shape=(28,28,1)
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape,name='input'),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3),kernel_initializer=tf.keras.initializers.zeros(),
                               activation="relu",use_bias=True,name='conv1'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="relu",use_bias=True,bias_initializer='zeros',name='dense1'),
        tf.keras.layers.Dense(1, activation="softmax",name='output'),
    ]
)

In [47]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 26, 26, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense1 (Dense)               (None, 10)                216330    
_________________________________________________________________
output (Dense)               (None, 1)                 11        
Total params: 216,661
Trainable params: 216,661
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.layers[2].get_weights()[0].shape

(21632, 10)

In [81]:
model = Sequential([
            # Input will be a vector of game_state_size length
            Dense(units=64, input_shape=(145,), use_bias=False, activation='relu'),
            Dense(units=16, use_bias=False , activation='relu'),
            Dense(units=20, use_bias=False , activation='sigmoid')
        ])

In [96]:
model.layers[0].get_weights()

[array([[ 0.15846945,  0.06074631, -0.01116447, ...,  0.02850984,
          0.10836868,  0.09538428],
        [-0.00508413,  0.01548098,  0.09077625, ..., -0.13243306,
         -0.07646503,  0.153733  ],
        [-0.06557158, -0.05744778, -0.01121141, ..., -0.06366156,
         -0.14667942,  0.05371822],
        ...,
        [-0.13363801, -0.13073698, -0.13653584, ..., -0.12998056,
         -0.07833579, -0.11672892],
        [-0.03898995, -0.09688929,  0.06942947, ..., -0.06940912,
          0.11978547,  0.05719483],
        [ 0.09379824, -0.13047986,  0.02441622, ...,  0.16348226,
          0.16421558,  0.14800413]], dtype=float32)]

In [73]:
np.random.rand(2,3)

array([[0.97386815, 0.26938907, 0.41723122],
       [0.0266759 , 0.106396  , 0.81905987]])

In [112]:
A = np.random.rand(2,3)
A.resize(6)
B = A[2:6]
B.resize(2,2);B

array([[0.76638923, 0.74973705],
       [0.56472813, 0.94235854]])

In [135]:
game_a = HanabiGame()
game_b = game_a
game_b.boardState(0)

[0,
 3,
 8,
 40,
 [-1, -1, -1, -1, -1],
 [[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]],
 [[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0]],
 [[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0]]]

In [136]:
game_b.performMove(index=0)
game_a.boardState(1)

[0,
 2,
 8,
 39,
 [-1, -1, -1, -1, -1],
 [[0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]],
 [[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 0]],
 [[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 1],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 1],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 1],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 1],
  [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 1]]]

In [133]:
game_b.legal_move_list

[(0, 0, -1, -1, -1),
 (0, 1, -1, -1, -1),
 (0, 2, -1, -1, -1),
 (0, 3, -1, -1, -1),
 (0, 4, -1, -1, -1),
 (1, 0, -1, -1, -1),
 (1, 1, -1, -1, -1),
 (1, 2, -1, -1, -1),
 (1, 3, -1, -1, -1),
 (1, 4, -1, -1, -1),
 (2, -1, 1, 0, -1),
 (2, -1, 1, 1, -1),
 (2, -1, 1, 2, -1),
 (2, -1, 1, 3, -1),
 (2, -1, 1, 4, -1),
 (2, -1, 1, -1, 0),
 (2, -1, 1, -1, 1),
 (2, -1, 1, -1, 2),
 (2, -1, 1, -1, 3),
 (2, -1, 1, -1, 4)]